In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from keras import backend as K
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os 
import multiprocessing
import wandb
# !pip install wandb -qqq
from wandb.keras import WandbCallback
import kerastuner as kt #!python3.x -m pip install keras-tuner
import cv2
from ipywidgets import fixed, interact 
import ipywidgets
from albumentations import (
    Compose, HorizontalFlip, CLAHE, HueSaturationValue,
    RandomBrightness, RandomContrast, RandomGamma,
    ToFloat, ShiftScaleRotate, RandomBrightnessContrast, RandomCrop)

In [ ]:
class Dataset_Classification(object): 
    """
     A dataset class usefull when training a classification model. 
    """
    def __init__(self, config): 
        self.config = config
        
        # set labels in a usefull format
        self._classification_labels()
        
        # initialize
        self.initialize()
        
        # to keep track of sampling
        self.sampling_check = np.zeros(20)
        self.times_sampled = 0
        
        
    def initialize(self):
        """
            Performs necessary thingss
        """
        self.train_data_path = 'data/train/img'
        self.test_data_path = 'data/test/img'
        
        # count nbr of files within data set. 
        self.nbr_of_train_images = len(os.listdir(self.train_data_path))
        self.nbr_of_test_images = len(os.listdir(self.test_data_path))
        
        # prepare train/validation split 
        train_fraction = self.config['train_fraction']
        r_idx=np.random.permutation(self.nbr_of_train_images)
        
        self.train_indices = r_idx[:int(train_fraction*self.nbr_of_train_images)]
        self.train_sample_probs = self.probabilities[self.train_indices]/np.sum(self.probabilities[self.train_indices])
        
        self.validation_indices = r_idx[int(train_fraction*self.nbr_of_train_images):]
        self.validation_sample_probs = self.probabilities[self.validation_indices]/np.sum(self.probabilities[self.validation_indices])
        
        print('Found {} train images'.format(self.nbr_of_train_images))
        print('- {} used for training, {} used for validating'.format(len(self.train_indices), len(self.validation_indices)))
        print('Found {} test images'.format(self.nbr_of_test_images))
        
        if self.config['augmentation']: 
            print('Including augmentation when training data is generated')
        self.augment = Compose([
                        #RandomCrop(width=self.config['input_shape'][0], height=self.config['input_shape'][0]),
                        HorizontalFlip(p=0.5),
                        RandomContrast(limit=0.1,p=0.25),
                        #RandomGamma(gamma_limit=(80, 120), p=0.5),
                        RandomBrightness(limit=0.15, p=0.5),
#                         HueSaturationValue(hue_shift_limit=1.5, sat_shift_limit=5,
#                                            val_shift_limit=2.5, p=.7),
                        # CLAHE(p=1.0, clip_limit=2.0),
                        ShiftScaleRotate(
                            shift_limit=0.1, scale_limit=0.1, 
                            rotate_limit=15, border_mode=cv2.BORDER_REFLECT_101, p=0.8), 
                    ])
    
        self.preprocessor = lambda x: x #defualt preprocessor does nothing 
        
    def get_test_set_size(self): 
        return self.nbr_of_test_images
    def get_train_set_size(self): 
        return self.nbr_of_train_images
        
    def reshape(self,im): 
        return cv2.resize(im, self.config['input_shape'])
        
    def _classification_labels(self): 
        """
            Get the classification labels 
        """
        # label names 
        train_df = pd.read_csv('data/train/train_set.csv', index_col="Id")
        self.label_names = train_df.columns.to_numpy()
        
        # get rid of pandas frame 
        self.labels = train_df.to_numpy() # each row corresponds to label
        
        # instances per class
        absolute_nbr_of_instances_per_class = np.sum(self.labels,axis=0)
        # get array with label name for each image 
        self.class_name_per_image = list()
        probabilities = list()
        total_class_prob = 1/20*np.ones(20)
        #total_class_prob[14]=0
        for row in self.labels: #can probably be done more eligant
            idx = np.where(row==1)[0]
            if 14 in idx:
                idx=14
            else:
                idx=idx[0]
            probabilities.append(total_class_prob[idx]/absolute_nbr_of_instances_per_class[idx])
            self.class_name_per_image.append(self.label_names[idx])
        
        
        # make sure total probability sums to 1
        if self.config['uniform_sample_probabilities']:
            self.probabilities = np.ones(np.array(probabilities).shape)/len(probabilities) # uniform sampling
        else:
            self.probabilities = np.array(probabilities)/np.sum(probabilities) # sampling based on distribution of classes in trainning data
            
            
    def feed_preprocess_function(self, preprocessor): 
        """
            Each network needs it's batches preprocessed in some manner. Feed this function to the Dataset object 
            who will call it when asking for batches.
            
            The preprocessor takes 
        """
        self.preprocessor = preprocessor
    
    def prepare_image(self, image): 
        """
            Function that performs all necessary steps from input image to image passed during trainnig. 
            This method should be overwritten depending on the model used.
        """
        h,w,c=image.shape
        # resize manually, when augmentation is turned on a random crop will be done 100% of times.
        #if not self.config['augmentation'] or h < self.config['input_shape'][0]or w < self.config['input_shape'][1]:
        image = self.reshape(image)

        # augment if augmentation is turned on 
        if self.config['augmentation']:
#             print('Augmentation enabled: check if combination of preprocessor and augmentation makes sence')
            image=self.augment(image=image)["image"]
        
        # preprocess 
        image = self.preprocessor(image)
        return image
    
    def prepare_test_image(self, image): 
        """
             Same as prepare_image but without augmentation
        """
        h,w,c=image.shape
        # resize manually, when augmentation is turned on a random crop will be done 100% of times.
        #if not self.config['augmentation'] or h < self.config['input_shape'][0]or w < self.config['input_shape'][1]:
        image = self.reshape(image)
        
        # preprocess 
        image = self.preprocessor(image)
        return image
    
    def view_preprocessed_image(self, image_id, option='train'): 
        """
            Shows an image as it is passed during training/testing of the network. 
            image_id 
        """
        assert hasattr(self, 'preprocessor'), 'set a preprocessor function before using this.'
        
        # get image 
        if option=='train':
            # get image 
            real_image = np.load('data/train/img/train_{}.npy'.format(image_id) )
            label = self.class_name_per_image[image_id]
        else: 
            real_image = np.load('data/test/img/test_{}.npy'.format(image_id) )
            label = 'unknown'
            
        image=np.copy(real_image)
        
        image = self.prepare_image(image)
        
    
    
        # print some info 
        print('original image:')
        print('-original_shape:', real_image.shape)
        print('-dtype:', real_image.dtype)
        print('-min value:', np.min(real_image))
        print('-max value:', np.max(real_image))
        
        print('final image:')
        print('-final shape:', image.shape)
        print('-dtype:', image.dtype)
        print('-min value:', np.min(image))
        print('-max value:', np.max(image))
        
        
        # show figure 
        fig, axes = plt.subplots(1,2, figsize=(30,15))
        axes[0].imshow(real_image)
        axes[0].set_title('Original image', fontsize=50)
        
        axes[1].imshow(image) # clip it to [0,1] range
        axes[1].set_title('preprocessed_image', fontsize=50)
        
        plt.suptitle('label: {}'.format(label), fontsize=50)
        fig.show()
        
    def view_possible_augmentations(self, image_id): 
        fig, axes = plt.subplots(4,4, figsize=(60,30))
        real_image = np.load('data/train/img/train_{}.npy'.format(image_id) )
        plt.suptitle('original image is on the top left', fontsize=50)
        for ax in axes.flat: 
            ax.imshow(self.augment(image=real_image)["image"])
            ax.axis('off')
        axes[0,0] = plt.imshow(real_image)
        fig.show()
        
        
    
    def train_generator(self,batch_size):
        """
            generator that will feed training batches during training 
        """
        inputs = []
        targets = []
        batchcount = 0
        while True:
#             for image_id in self.train_indices:
            image_id = np.random.choice(self.train_indices, p=self.train_sample_probs)
            # sample real image
            real_image = np.load('data/train/img/train_{}.npy'.format(image_id) )

            image = self.prepare_image(np.copy(real_image))
            inputs.append(image)

            # get corresponding label 
            targets.append(self.labels[image_id])
            
            self.sampling_check+=self.labels[image_id]
            self.times_sampled+=1
            

            batchcount += 1
            if batchcount >= batch_size:
                X = np.array(inputs)
                y = np.array(targets, dtype=np.uint8)
                yield (X, y)
                inputs = []
                targets = []
                batchcount = 0

    def validation_generator(self,batch_size):
        """
            generator that will feed validation batches during training 
        """
        inputs = []
        targets = []
        batchcount = 0
        while True:
            #for image_id in self.validation_indices:
            # sample real image
            image_id = np.random.choice(self.validation_indices,p=self.validation_sample_probs)
            real_image = np.load('data/train/img/train_{}.npy'.format(image_id) )

            image = self.prepare_test_image(np.copy(real_image))

            inputs.append(image)

            # get corresponding label 
            targets.append(self.labels[image_id])

            batchcount += 1
            if batchcount >= batch_size:
                X = np.array(inputs)
                y = np.array(targets, dtype=np.uint8)
                yield (X, y)
                inputs = []
                targets = []
                batchcount = 0           
    
    def test_generator(self,batch_size):
        """
            generator for feeding test data to model for prediction
        """
        inputs = []
        img_id = 0
        while img_id < self.nbr_of_test_images:
            raw_image = np.load('data/test/img/test_{}.npy'.format(img_id))
            image = self.prepare_test_image(np.copy(raw_image))
            inputs.append(image)
            img_id += 1
            if img_id%batch_size == 0:
                X = np.array(inputs)
                yield X
                inputs = []
        if len(inputs) > 0:
            return np.array(inputs)
    
    def show_class_distribution(self): 
        fig,axes=plt.subplots(figsize=(30,15))
        class_probs=np.mean(self.labels, axis=0)
        axes.bar(self.label_names,  class_probs)
        axes.tick_params(axis='both', which='major', labelsize=30)
        for tick in axes.xaxis.get_major_ticks():
            tick.label.set_rotation('vertical')
        plt.suptitle('Class distribution within the training data.', fontsize=50)
        fig.show()
        
    def show_training_sampling_distribution(self): 
        fig,axes=plt.subplots(figsize=(30,15))
        class_probs=np.mean(self.labels, axis=0)
        axes.bar(self.label_names,  self.sampling_check/self.times_sampled)
        axes.tick_params(axis='both', which='major', labelsize=30)
        for tick in axes.xaxis.get_major_ticks():
            tick.label.set_rotation('vertical')
        plt.suptitle('Number of times each class was sampled during training.', fontsize=50)
        fig.show()
    
    def get_class_distribution(self):
        return np.mean(self.labels, axis=0)

In [ ]:
dataset_config = {
    'train_fraction': 0.9,
    'input_shape': (224, 224),
    'augmentation': True, 
    'uniform_sample_probabilities': False
}
ds = Dataset_Classification(dataset_config)
ds.show_class_distribution()

In [ ]:
stop=0
for (X,y) in ds.train_generator(10):
    stop+=1
    print(X.shape)
    print(y.shape)
    
    if stop>10: 
        break
    

In [ ]:
class RandomClassificationModel:
    """
    Random classification model: 
        - generates random labels for the inputs based on the class distribution observed during training
        - assumes an input can have multiple labels
    """
    def fit(self, X, y):
        """
        Adjusts the class ratio variable to the one observed in y. 

        Parameters
        ----------
        X: list of arrays - n x (height x width x 3)
        y: list of arrays - n x (nb_classes)

        Returns
        -------
        self
        """
        self.distribution = np.mean(y, axis=0)
        print("Setting class distribution to:\n{}".format("\n".join(f"{label}: {p}" for label, p in zip(labels, self.distribution))))
        return self
        
    def predict(self, X):
        """
        Predicts for each input a label.
        
        Parameters
        ----------
        X: list of arrays - n x (height x width x 3)
            
        Returns
        -------
        y_pred: list of arrays - n x (nb_classes)
        """
        np.random.seed(0)
        return [np.array([int(np.random.rand() < p) for p in self.distribution]) for _ in X]
    
    def __call__(self, X):
        return self.predict(X)
    
    
class ClassifactionModel(RandomClassificationModel): 
    """
        Main class implementing all functions necessary to train and/or use a classification model 
        This class has to be overwritten for each specific model of interest, where the base model should be implemented.
    """
    def __init__(self, config): 
        self.config = config 
        self.config_head = config['head_model']
        
        # initialize dataset
        self.dataset = Dataset_Classification(config['dataset'])
              
        # check if some configurations make sense 
        assert len(self.config_head['head_model_units']) == len(self.config_head['add_dropout']), 'head_models_units and add_dropout list should have same size'
    
    
    def set_config(self, config):
        self.config = config 
        self.config_head = config['head_model']
        self.dataset = Dataset_Classification(config['dataset'])
        assert len(self.config_head['head_model_units']) == len(self.config_head['add_dropout']), 'head_models_units and add_dropout list should have same size'
        
    def predict(self, X):
        # 
        
        if len(X.shape) == 1: 
            # X is a batch of images prepare all of them and create batch. 
            batch = np.array([self.dataset.prepare_test_image(im) for im in X])
            y = model.predict(batch)
        else: 
            # X is a single image 
            batch = self.dataset.prepare_test_image(X)
            batch = np.expand_dims(batch, axis=0)
            y = model.predict(batch)
            
        y = np.squeeze(y)

        label_idx = np.where(y==1.)

        return self.dataset.label_names[label_idx]
            
    def build(self): 
        """
            Builds the model 
        """
#       self.base_model = resnet50
        
        # define a head model
        head_model=keras.layers.GlobalAveragePooling2D()(self.base_model.output)

        for (nbr_units, dropout) in zip(self.config_head['head_model_units'], self.config_head['add_dropout']): 
            head_model=tf.keras.layers.Dense(nbr_units, activation=self.config_head['activation'])(head_model)
            if dropout:
                head_model=tf.keras.layers.Dropout(0.4)(head_model)
        
        head_model=keras.layers.Dense(20, activation='softmax')(head_model)
        #self.config['nbr_classes']
        self.head_model = head_model
                  
        # combine both models 
        self.model = keras.Model(self.base_model.input, head_model)


#         avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
#         output = keras.layers.Dense(20, activation='softmax')(avg)
#         model=keras.Model(inputs=base_model.input, outputs=output)
           
    def compile_model(self): 
        # optimizer
        if self.config['train_parameters']['optimizer'] == 'SGD':
            optimizer = tf.keras.optimizers.SGD(
                    learning_rate=self.config['train_parameters']['learning_rate'], momentum=0.9,
                    nesterov=False, name="SGD"
                )
        elif self.config['train_parameters']['optimizer'] == 'ADAM':
            optimizer = tf.keras.optimizers.Adam(lr=self.config['train_parameters']['learning_rate'])

        # metric
        metrics = [tf.keras.metrics.CategoricalAccuracy(),
                  tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top 3 categorical acccuracy'), 
                  tf.keras.metrics.TopKCategoricalAccuracy(k=5, name='top 5 categorical acccuracy')
                  ]
        
        # loss
        loss='categorical_crossentropy'
        self.model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
        
        
    def train(self, name_run, notes, tags):
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            try:
                # Currently, memory growth needs to be the same across GPUs
                for gpu in gpus:
                    tf.config.experimental.set_memory_growth(gpu, True)
                logical_gpus = tf.config.experimental.list_logical_devices('GPU')
                print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
            except RuntimeError as e:
                # Memory growth must be set before GPUs have been initialized
                print(e)
            
        # setup logging
        if self.config['logging_wandb']:
            # w&b 
            wandb.init(name=name_run, 
                   project=self.project_name,
                   notes=notes, 
                   tags=tags,
                   entity='cv-task-2')

            # save usefull config to w&b
            wandb.config.learning_rate = self.config['train_parameters']['learning_rate']
            wandb.config.batch_size = self.config['train_parameters']['batch_size']
            wandb.config.epochs = self.config['train_parameters']['epochs']
            wandb.config.steps_per_epoch = self.config['train_parameters']['steps_per_epoch']
             
        # build model 
        self.build()

        # set model parts trainable or not
        if self.config['train_base_model'] == False: 
            print('freezing base model layers')
            for layer in self.base_model.layers:
                layer.trainable = False
        if self.config['train_head_model'] == False: 
            print('freezing head model layers')
            for layer in self.head_model.layers:
                layer.trainable = False
        
        
        # compile model
        self.compile_model()
        
        if self.config['logging_wandb']:
            # set save_model true if you want wandb to upload weights once run has finished (takes some time)
            clbcks = [WandbCallback(save_model=False)]
        else: 
            clbcks = []

        
        # start training 
        history=self.model.fit(
                    x = self.dataset.train_generator(batch_size=self.config['train_parameters']['batch_size']),
                    steps_per_epoch = self.config['train_parameters']['steps_per_epoch'],
                    epochs=self.config['train_parameters']['epochs'], 
                    validation_data=self.dataset.validation_generator(batch_size=self.config['train_parameters']['batch_size']),
                    validation_steps=20, 
                    callbacks=clbcks
        )
        
        #workers=multiprocessing.cpu_count(),
        #use_multiprocessing=True,
    
    def prepare_for_inference(self, model_weights_path): 
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            try:
                # Currently, memory growth needs to be the same across GPUs
                for gpu in gpus:
                    tf.config.experimental.set_memory_growth(gpu, True)
                logical_gpus = tf.config.experimental.list_logical_devices('GPU')
                print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
            except RuntimeError as e:
                # Memory growth must be set before GPUs have been initialized
                print(e)
        self.build()
        self.model.load_weights(model_weights_path)
    
    def show_heatmap_prediction(self, image_id):
        LAYER_NAME=self.heatmap_layer_name
        im = np.load('data/test/img/test_{}.npy'.format(image_id))
        pre_im = self.dataset.prepare_test_image(im)
        batch = np.expand_dims(pre_im, axis=0)
    
        pred = self.model.predict(batch)
        idx=np.argmax(pred)
        score = np.round(pred[0][idx]/np.sum(pred),4)
        label=self.dataset.label_names[idx]

        grad_model = tf.keras.models.Model([self.model.inputs], [self.model.get_layer(LAYER_NAME).output, self.model.output])

        with tf.GradientTape() as tape:
            conv_outputs, predictions = grad_model(batch)
            loss = predictions[:, idx]

        output = conv_outputs[0]
        grads = tape.gradient(loss, conv_outputs)[0]

        gate_f = tf.cast(output > 0, 'float32')
        gate_r = tf.cast(grads > 0, 'float32')
        guided_grads = tf.cast(output > 0, 'float32') * tf.cast(grads > 0, 'float32') * grads

        weights = tf.reduce_mean(guided_grads, axis=(0, 1))

        cam = np.ones(output.shape[0: 2], dtype = np.float32)

        for i, w in enumerate(weights):
            cam += w * output[:, :, i]

        cam = cv2.resize(cam.numpy(), (224, 224))
        cam = np.maximum(cam, 0)
        heatmap = (cam - cam.min()) / (cam.max() - cam.min())

        cam = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)
        og_im = cv2.cvtColor(im.astype('uint8'), cv2.COLOR_RGB2BGR)

        og_im = cv2.resize(og_im, (224, 224))


        output_image = cv2.addWeighted(og_im, 0.7, cam, 1, 0)


        fig, axes = plt.subplots(1,2, figsize=(30,15))
        axes[1].imshow(cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB))
        axes[0].imshow(im)
        axes[0].set_title('prediction: {}, score: {}'.format(label, np.round(100*score,2)), fontsize=25)
        plt.show()

In [ ]:
class XceptionModel(ClassifactionModel): 
    def __init__(self, config):
        # setup model name for wandb
        self.project_name = 'Xception'
        self.heatmap_layer_name='block14_sepconv2_act'
        # define the base model
        self.base_model = keras.applications.xception.Xception(weights="imagenet",
                                                               include_top=False)
        # super takes care of the rest
        super().__init__(config)
        
        # feed preprocessor function 
        self.dataset.feed_preprocess_function(keras.applications.xception.preprocess_input)

In [ ]:
config = {
    'name': 'XceptionModel',
    'logging_wandb': False,  #nice tool for tracking a run. make and account on wandb.ai and I will add you to this project
    'weights': "imagenet", # 'imagenet', #None, 
    'nbr_classes': 20,
    'input_shape': (224, 224, 3),
    'train_base_model': True, # whether to train the head and or base model
    'train_head_model': True, 
    'train_parameters': {
        'optimizer': 'ADAM',
        'epochs': 5,
        'batch_size': 64,
        'learning_rate': 0.00001, 
        'steps_per_epoch': 2000
    },
    'dataset': {
        'train_fraction': 0.9,
        'input_shape': (224, 224),
        'augmentation': True, # whether to augment images or not
        'uniform_sample_probabilities': False
    },
    'head_model': {
        'head_model_units': [], 
        'add_dropout':      [],
        'activation': 'relu'
    }
}

In [ ]:
Xception = XceptionModel(config)
Xception.prepare_for_inference('weights/Xception_finetuned.h5')

In [ ]:
Xception.show_heatmap_prediction(50)

In [ ]:
class ResNet50Model(ClassifactionModel): 
    def __init__(self, config):
        # setup model name for wandb
        self.project_name = 'resnet50'
        self.heatmap_layer_name='conv5_block3_out'
        # define the base model
        self.base_model = keras.applications.ResNet50V2(weights="imagenet",
                                                               include_top=False)
        # super takes care of the rest
        super().__init__(config)
        
        # feed preprocessor function 
        self.dataset.feed_preprocess_function(keras.applications.resnet_v2.preprocess_input)

In [ ]:
name_run='ResNet50_firstTry'
notes='First try for finetuning resnet50 on pretrained imagenet weights. Data augmentation turned on. '
tags = ['resnet50', 'head = []', 'head = []', 'Augmentation applied', 'uniform class distribution']
resnet50=ResNet50Model(config)
resnet50.train(name_run, notes, tags)

In [ ]:
resnet50=ResNet50Model(config)
resnet50.prepare_for_inference('weights/resnet50.h5')

In [ ]:
resnet50.show_heatmap_prediction(50)

In [ ]:
resnet50.model.get_layer('conv5_block3_out').output